In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, f1_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer, load_wine
from ucimlrepo import fetch_ucirepo
import torch
import sys
import openml
sys.path.append('..')
from idistill.whitebox_figs import FIGSRegressorCV, FIGSRegressor


/home/mattyshen/anaconda3/lib/python3.9/site-packages/aiohttp/helpers.py:107: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def noop(*args, **kwargs):  # type: ignore


In [2]:
import pickle

In [3]:
with open('cebab_mlp2_tbm.pkl', 'rb') as f:
    data = pickle.load(f)

In [4]:
X_train_t = data['X_train_t']
y_train_t = data['y_train_t']
X_test_t = data['X_test_t']
y_test_t = data['y_test_t']
y_train = data['y_train']
y_test = data['y_test']

In [5]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

# Fit and transform the data
encoded_data = encoder.fit_transform(X_train_t)

# Convert the result to a DataFrame
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out())
encoded_df

,Food Quality_-2.0,Food Quality_0.0,Food Quality_1.0,Food Quality_2.0,service quality_-2.0,service quality_-1.0,service quality_0.0,service quality_1.0,Price Perceptions_-1.0,Price Perceptions_0.0,...,Server Attentiveness_0.0,Server Attentiveness_0.5,Server Attentiveness_1.0,Unique Dining Experience_0.0,Unique Dining Experience_1.0,Food Arrival Timing_-2.0,Food Arrival Timing_-1.0,Food Arrival Timing_0.0,Food Arrival Timing_1.0,Food Arrival Timing_2.0
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1459,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1460,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1461,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
figs = FIGSRegressor(max_rules = 20, max_trees = 5, max_depth = 3)
figs.fit(encoded_df, y_train_t)

FIGSRegressor(max_depth=3, max_rules=20, max_trees=5)

In [7]:
len(figs.trees_), figs.complexity_

(5, 19)

In [8]:
encoder.transform(X_test_t).toarray().shape

(250, 68)

In [9]:
mean_squared_error(y_test, figs.predict(encoder.transform(X_test_t).toarray()))

0.2920119073297879

In [10]:
r2_score(y_train_t, figs.predict(encoded_df)), r2_score(y_test_t, figs.predict(encoder.transform(X_test_t).toarray()))

(0.9784874728542855, 0.9798345491589026)

In [11]:
mean_squared_error(y_test, figs.predict(X_test_t > 0))

IndexError: index 26 is out of bounds for axis 0 with size 18

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape, X_test.shape

In [ ]:
# Initialize a classifier
clf = TabPFNClassifier(ignore_pretraining_limits=True)
clf.fit(X_train, y_train)

# # Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

In [ ]:
y_train_preds = clf.predict_proba(X_train)

In [ ]:
accuracy_score(np.argmax(y_train_preds, axis = 1), y_train)

In [ ]:
figs = FIGSRegressor(max_rules = 100, max_trees = 20, max_depth = 3, min_impurity_decrease=0.0001)
x = xgb.XGBRegressor(n_estimators = 20, max_depth=3)
figs.fit(X_train, y_train_preds)
x.fit(X_train, y_train_preds)

In [ ]:
figs.complexity_, len(figs.trees_)

In [ ]:
accuracy_score(np.argmax(y_train_preds, axis = 1), np.argmax(figs.predict(X_train), axis = 1))

In [ ]:
accuracy_score(y_train, np.argmax(figs.predict(X_train), axis = 1))

In [ ]:
accuracy_score(y_test, np.argmax(figs.predict(X_test), axis = 1))

In [ ]:
accuracy_score(predictions, np.argmax(figs.predict(X_test), axis = 1))

In [ ]:
#xgb
accuracy_score(np.argmax(y_train_preds, axis = 1), np.argmax(x.predict(X_train), axis = 1))

In [ ]:
accuracy_score(y_train, np.argmax(x.predict(X_train), axis = 1))

In [ ]:
accuracy_score(y_test, np.argmax(x.predict(X_test), axis = 1))

In [ ]:
accuracy_score(predictions, np.argmax(x.predict(X_test), axis = 1))

In [ ]:
num_rules = 0
trees = x.get_booster().get_dump(with_stats=True)

for tree in trees:
    num_rules += len([line for line in tree.splitlines() if 'leaf=' not in line])

print(num_rules, figs.complexity_/num_rules)

In [ ]:
figs.complexity_, len(figs.trees_)

In [ ]:
clf.predict_proba(X_train)